In [29]:
import string
import pandas as pd
import numpy as np
import torch
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# nltk.download('wordnet')

In [37]:
data = 'data/WASSA23_conv_level_with_labels_train.tsv'
df = pd.read_table(data, header=0)
new_col = []
for names in df.columns:
    new_col.append(names.strip())
df.columns = new_col
dataset = df.drop(["conversation_id", "turn_id", "speaker_number", "article_id", "speaker_id", "essay_id"], axis=1, inplace=True)

X_data, y_data = df.loc[:, 'text'], df.drop('text', axis=1)
X_train, X_test, y_train , y_test = train_test_split(X_data, y_data, train_size=0.8)
#reset index of training examples
X_train, X_test = X_train.reset_index(drop=True), X_test.reset_index(drop=True)
y_train, y_test = y_train.reset_index(drop=True), y_test.reset_index(drop=True)

- tokenization
- remove stop word and punctuatuons, numbers
- lematization
- vectorization

In [38]:
def word_preprocessor(sentence):
    stop_words = set(stopwords.words('english'))
    punctuations = set(string.punctuation)
    lem = WordNetLemmatizer().lemmatize
    sentence = word_tokenize(sentence)
    sentence = [word for word in sentence if word not in stop_words]
    sentence = [word for word in sentence if word not in punctuations]
    sentence_str = ' '.join(sentence)
    sentence = lem(sentence_str)
    return sentence #sentence

In [39]:
X_train = X_train.apply(word_preprocessor)
X_test = X_test.apply(word_preprocessor)

#convert labels to array
X_train, X_test = np.array(X_train), np.array(X_test)
y_train, y_test = np.array(y_train[['EmotionalPolarity', 'Emotion', 'Empathy']]), np.array(y_test[['EmotionalPolarity', 'Emotion', 'Empathy']])
y_train, y_test


(array([[0.6667, 2.    , 2.    ],
        [2.    , 2.    , 3.    ],
        [1.    , 1.3333, 0.6667],
        ...,
        [1.3333, 2.    , 1.6667],
        [2.    , 4.3333, 2.6667],
        [1.    , 0.6667, 0.3333]]),
 array([[0.    , 2.3333, 2.3333],
        [1.3333, 1.6667, 1.6667],
        [1.3333, 1.    , 2.6667],
        ...,
        [0.    , 2.    , 1.3333],
        [1.3333, 2.    , 1.6667],
        [1.6667, 2.    , 2.    ]]))

In [33]:
vectorizer = CountVectorizer(max_features=3000, stop_words='english', lowercase=True)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [34]:
X_train_vec = X_train_vec.toarray()
X_test_vec = X_test_vec.toarray()

In [49]:
# Create a sequential model
EEPD_Model = Sequential()
EEPD_Model.add(Dense(100, activation='relu', input_dim=X_train_vec.shape[1]))
EEPD_Model.add(Dense(80, activation='relu'))
EEPD_Model.add(Dense(50, activation='relu'))
EEPD_Model.add(Dense(3, activation='softmax'))

# Compile the model
EEPD_Model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Train the model
EEPD_Model.fit(X_train_vec, y_train, epochs=20, batch_size=100, validation_data=(X_test_vec, y_test))

# Evaluate the model
loss, accuracy = EEPD_Model.evaluate(X_test_vec, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Epoch 1/50
71/71 [==============================] - 3s 18ms/step - loss: 2.9584 - accuracy: 0.5581 - val_loss: 2.9940 - val_accuracy: 0.5757
Epoch 2/50
71/71 [==============================] - 1s 9ms/step - loss: 2.9193 - accuracy: 0.5748 - val_loss: 2.9979 - val_accuracy: 0.5262
Epoch 3/50
71/71 [==============================] - 1s 10ms/step - loss: 2.9015 - accuracy: 0.6330 - val_loss: 3.0059 - val_accuracy: 0.4812
Epoch 4/50
71/71 [==============================] - 1s 8ms/step - loss: 2.8813 - accuracy: 0.6796 - val_loss: 3.0154 - val_accuracy: 0.5171
Epoch 5/50
71/71 [==============================] - 1s 9ms/step - loss: 2.8668 - accuracy: 0.7246 - val_loss: 3.0216 - val_accuracy: 0.5011
Epoch 6/50
71/71 [==============================] - 1s 12ms/step - loss: 2.8593 - accuracy: 0.7348 - val_loss: 3.0232 - val_accuracy: 0.4989
Epoch 7/50
71/71 [==============================] - 1s 11ms/step - loss: 2.8556 - accuracy: 0.7484 - val_loss: 3.0243 - val_accuracy: 0.4795
Epoch 8/50
71/71

In [106]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((7020,), (7020, 3), (1756,), (1756, 3))